# PROG74040-24S-Sec1
# Final Project
# Eazaz Jakda, Navdeep Mangat, Rachel Le
# August 12, 2024

In [ ]:
!pip install keras-preprocessing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.4 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import numpy as np
import re
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, classification_report, confusion_matrix, f1_score, accuracy_score, ConfusionMatrixDisplay
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, Attention, Input
from tensorflow.keras.models import Model
import chardet
import sys
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D

In [ ]:
import tensorflow as tf
if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. Neural nets can be very slow without a GPU.")
    if "google.colab" in sys.modules:
        print("Go to Runtime > Change runtime and select a GPU hardware "
              "accelerator.")
    if "kaggle_secrets" in sys.modules:
        print("Go to Settings > Accelerator and select GPU.")

Inital read-in to obtain the encoding value

In [ ]:
# with open('/content/training.1600000.processed.noemoticon.csv', 'rb') as file:
#     result = chardet.detect(file.read())
#     encoding = result['encoding']

# data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding=encoding)

# data

Can now use this encoding value to read in the dataset like typical CSV files by specifying the encoding.

In [ ]:
#This is the encoder that we got from the code above, needed to read in the dataset
encoder = "Windows-1252"

data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding = encoder)


# Pre-processing the Data

In [ ]:
#Adding column names as the dataaset didn't have any.
data.columns = ["encoded_sentiment", "time", "date", "query", "username", "text"]

#Dropping the columns that aren't needed
data.drop(columns=["time", "date", "query", "username"], inplace=True)

#Dataset was using 0 and 4 for the sentiment encoding, changing the 4 to 1
data['encoded_sentiment'][data['encoded_sentiment']==4]=1

data

<ipython-input-8-b827fffd8e61>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['encoded_sentiment'][data['encoded_sentiment']==4]=1


,encoded_sentiment,text
0,0,is upset that he can't update his Facebook by ...
1,0,@Kenichan I dived many times for the ball. Man...
2,0,my whole body feels itchy and like its on fire
3,0,"@nationwideclass no, it's not behaving at all...."
4,0,@Kwesidei not the whole crew
...,...,...
1599994,1,Just woke up. Having no school is the best fee...
1599995,1,TheWDB.com - Very cool to hear old Walt interv...
1599996,1,Are you ready for your MoJo Makeover? Ask me f...
1599997,1,Happy 38th Birthday to my boo of alll time!!! ...


In [ ]:
#preprocessing function to remove punctuation and special characters
def text_preprocessing(text):
    try:
      text = text.lower()
      text = re.sub(r'[^a-z0-9\s]', '', text)
    except:
        print(text)
    return text

#Applying preprocessing function to text column
data_processed = data.copy()
data_processed['text'] = data_processed['text'].apply(text_preprocessing)

#Sampling the dataset to use 10% of the data
data_processed = data_processed.sample(frac=0.1).reset_index(drop=True)

data_processed.head()

,encoded_sentiment,text
0,0,and its gone all over my bag
1,0,tolaniwatch where are you im waiting for the s...
2,0,minijesus u just said the igoogle apps were crap
3,0,oh no have a very poorly pussy cat not good
4,0,i also just changed alices dads cars oil to he...


In [ ]:
#Initial number of tokens, sentence length and embedding layer parameter
max_words = 2000
max_len = 50
embedding_dim = 32

#Tokenizing the text column
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data_processed['text'])
sequences = tokenizer.texts_to_sequences(data_processed['text'])

#Padding sequences to be same length
X = pad_sequences(sequences, maxlen=max_len)
y = np.array(data_processed['encoded_sentiment'])

## Experiment 1

Basically a bare-minimum LSTM network

In [ ]:
#Input layer
input_layer = Input(shape=(max_len,))

#Embedding layer
embedding_layer = Embedding(max_words, embedding_dim, input_length=max_len)(input_layer)

#LSTM Layer
lstm_layer = LSTM(64, return_sequences=False)(embedding_layer)

#Dropout Layer
dropout_layer = Dropout(0.5)(lstm_layer)

#Dense Layers
dense_layer = Dense(16, activation='relu')(dropout_layer)
output_layer = Dense(2, activation='softmax')(dense_layer)

#Building the model
model = Model(inputs=input_layer, outputs=output_layer)

#Compiling the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Training the model
model.fit(X, y, epochs=10, batch_size=16, validation_split=0.2)

#Showing the Model summary
model.summary()

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


8000/8000 ━━━━━━━━━━━━━━━━━━━━ 52s 6ms/step - accuracy: 0.7304 - loss: 0.5298 - val_accuracy: 0.7806 - val_loss: 0.4616
Epoch 2/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 83s 6ms/step - accuracy: 0.7849 - loss: 0.4539 - val_accuracy: 0.7828 - val_loss: 0.4525
Epoch 3/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 50s 6ms/step - accuracy: 0.7948 - loss: 0.4390 - val_accuracy: 0.7870 - val_loss: 0.4511
Epoch 4/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 82s 6ms/step - accuracy: 0.8024 - loss: 0.4247 - val_accuracy: 0.7869 - val_loss: 0.4517
Epoch 5/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 51s 6ms/step - accuracy: 0.8076 - loss: 0.4155 - val_accuracy: 0.7878 - val_loss: 0.4549
Epoch 6/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 82s 6ms/step - accuracy: 0.8140 - loss: 0.4043 - val_accuracy: 0.7898 - val_loss: 0.4515
Epoch 7/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 83s 7ms/step - accuracy: 0.8198 - loss: 0.3954 - val_accuracy: 0.7891 - val_loss: 0.4537
Epoch 8/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 86s 7ms/step - accuracy: 0.8254 - loss: 0.3827 - val

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)           │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_2 (Embedding)              │ (None, 50, 32)              │          64,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 64)                  │          24,832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 64)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 16)                  │           1,040 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 2)                   │              34 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 269,720 (1.03 MB)

 Trainable params: 89,906 (351.20 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 179,814 (702.40 KB)

## Experiment 2

Integrate bidirectional layer

In [ ]:
#Input layer
input_layer = Input(shape=(max_len,))

#Embedding layer
embedding_layer = Embedding(max_words, embedding_dim, input_length=max_len)(input_layer)

#Bidirectional LSTM Layer
bidirectional_layer = Bidirectional(LSTM(64, return_sequences=False))(embedding_layer)

#Dropout Layer
dropout_layer = Dropout(0.5)(bidirectional_layer)

#Dense Layers
dense_layer = Dense(16, activation='relu')(dropout_layer)
output_layer = Dense(2, activation='softmax')(dense_layer)

#Building the model
model = Model(inputs=input_layer, outputs=output_layer)

#Compiling the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Training the model
model.fit(X, y, epochs=10, batch_size=16, validation_split=0.2)

#Showing the Model summary
model.summary()

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


8000/8000 ━━━━━━━━━━━━━━━━━━━━ 76s 9ms/step - accuracy: 0.7261 - loss: 0.5315 - val_accuracy: 0.7807 - val_loss: 0.4601
Epoch 2/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 78s 10ms/step - accuracy: 0.7847 - loss: 0.4535 - val_accuracy: 0.7859 - val_loss: 0.4546
Epoch 3/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 73s 9ms/step - accuracy: 0.7956 - loss: 0.4359 - val_accuracy: 0.7863 - val_loss: 0.4480
Epoch 4/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 83s 9ms/step - accuracy: 0.8047 - loss: 0.4224 - val_accuracy: 0.7872 - val_loss: 0.4484
Epoch 5/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 86s 10ms/step - accuracy: 0.8099 - loss: 0.4146 - val_accuracy: 0.7866 - val_loss: 0.4483
Epoch 6/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 79s 9ms/step - accuracy: 0.8181 - loss: 0.3994 - val_accuracy: 0.7871 - val_loss: 0.4519
Epoch 7/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 82s 9ms/step - accuracy: 0.8217 - loss: 0.3903 - val_accuracy: 0.7860 - val_loss: 0.4593
Epoch 8/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 81s 9ms/step - accuracy: 0.8293 - loss: 0.3777 - v

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)           │ (None, 50)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ embedding_4 (Embedding)              │ (None, 50, 32)              │          64,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 128)                 │          49,664 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 16)                  │           2,064 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 2)                   │              34 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 347,288 (1.32 MB)

 Trainable params: 115,762 (452.20 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 231,526 (904.40 KB)

## Experiment 3

Integrate attention mechanisms and multiple layers

In [ ]:
#Input layer
input_layer = Input(shape=(max_len,))

#Embedding layer
embedding_layer = Embedding(max_words, embedding_dim, input_length=max_len)(input_layer)

#Bidirectional LSTM Layer
bidirectional_layer = Bidirectional(LSTM(64, return_sequences=True))(embedding_layer)

#Attention Layer
attention_layer = Attention()([bidirectional_layer, bidirectional_layer])

#Dropout Layer
dropout_layer1 = Dropout(0.5)(attention_layer)

#LSTM Layer
lstm_layer2 = LSTM(32)(dropout_layer1)

#Dropout Layer
dropout_layer2 = Dropout(0.5)(lstm_layer2)

#Dense Layers
dense_layer = Dense(16, activation='relu')(dropout_layer2)
output_layer = Dense(2, activation='softmax')(dense_layer)

#Building the model
model = Model(inputs=input_layer, outputs=output_layer)

#Compiling the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Training the model
model.fit(X, y, epochs=10, batch_size=16, validation_split=0.2)

#Showing the Model summary
model.summary()

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


8000/8000 ━━━━━━━━━━━━━━━━━━━━ 108s 13ms/step - accuracy: 0.6778 - loss: 0.5899 - val_accuracy: 0.7764 - val_loss: 0.4698
Epoch 2/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 99s 12ms/step - accuracy: 0.7818 - loss: 0.4675 - val_accuracy: 0.7860 - val_loss: 0.4553
Epoch 3/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 144s 13ms/step - accuracy: 0.7946 - loss: 0.4429 - val_accuracy: 0.7872 - val_loss: 0.4507
Epoch 4/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 142s 13ms/step - accuracy: 0.8006 - loss: 0.4306 - val_accuracy: 0.7878 - val_loss: 0.4504
Epoch 5/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 141s 12ms/step - accuracy: 0.8053 - loss: 0.4190 - val_accuracy: 0.7879 - val_loss: 0.4479
Epoch 6/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 141s 12ms/step - accuracy: 0.8105 - loss: 0.4115 - val_accuracy: 0.7871 - val_loss: 0.4502
Epoch 7/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 142s 12ms/step - accuracy: 0.8188 - loss: 0.3979 - val_accuracy: 0.7888 - val_loss: 0.4487
Epoch 8/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 143s 13ms/step - accuracy: 0.8237 - los

Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_7             │ (None, 50)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_7 (Embedding)   │ (None, 50, 32)         │         64,000 │ input_layer_7[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_4           │ (None, 50, 128)        │         49,664 │ embedding_7[0][0]      │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_2 (Attention)   │ (None, 50, 128)        │              0 │ bidirectional_4[0][0], │
│                           │                        │                │ bidirectional_4[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_7 (Dropout)       │ (None, 50, 128)        │              0 │ attention_2[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_10 (LSTM)            │ (None, 32)             │         20,608 │ dropout_7[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_8 (Dropout)       │ (None, 32)             │              0 │ lstm_10[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_10 (Dense)          │ (None, 16)             │            528 │ dropout_8[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_11 (Dense)          │ (None, 2)              │             34 │ dense_10[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 404,504 (1.54 MB)

 Trainable params: 134,834 (526.70 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 269,670 (1.03 MB)

## Experiment 4

Add convolutional layer

In [ ]:
#Input layer
input_layer = Input(shape=(max_len,))

#Embedding layer
embedding_layer = Embedding(max_words, embedding_dim, input_length=max_len)(input_layer)

#Convolutional Layer
conv_layer = Conv1D(filters=128, kernel_size=3, activation='relu')(embedding_layer)

#Bidirectional LSTM Layer
bidirectional_layer = Bidirectional(LSTM(64, return_sequences=True))(conv_layer)

#Attention Layer
attention_layer = Attention()([bidirectional_layer, bidirectional_layer])

#Dropout Layer
dropout_layer1 = Dropout(0.5)(attention_layer)

#LSTM Layer
lstm_layer2 = LSTM(32)(dropout_layer1)

#Dropout layer
dropout_layer2 = Dropout(0.5)(lstm_layer2)

#Dense Layers
dense_layer = Dense(16, activation='relu')(dropout_layer2)
output_layer = Dense(2, activation='softmax')(dense_layer)

#Building the model
model = Model(inputs=input_layer, outputs=output_layer)

#Compiling the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Training the model
model.fit(X, y, epochs=10, batch_size=16, validation_split=0.2)

#Showing the Model summary
model.summary()

Epoch 1/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 108s 13ms/step - accuracy: 0.7175 - loss: 0.5419 - val_accuracy: 0.7803 - val_loss: 0.4647
Epoch 2/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 142s 13ms/step - accuracy: 0.7894 - loss: 0.4528 - val_accuracy: 0.7831 - val_loss: 0.4552
Epoch 3/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 142s 13ms/step - accuracy: 0.8007 - loss: 0.4332 - val_accuracy: 0.7879 - val_loss: 0.4504
Epoch 4/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 142s 13ms/step - accuracy: 0.8113 - loss: 0.4144 - val_accuracy: 0.7879 - val_loss: 0.4503
Epoch 5/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 142s 13ms/step - accuracy: 0.8247 - loss: 0.3920 - val_accuracy: 0.7902 - val_loss: 0.4522
Epoch 6/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 141s 13ms/step - accuracy: 0.8324 - loss: 0.3751 - val_accuracy: 0.7836 - val_loss: 0.4661
Epoch 7/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 142s 13ms/step - accuracy: 0.8423 - loss: 0.3565 - val_accuracy: 0.7850 - val_loss: 0.4715
Epoch 8/10
8000/8000 ━━━━━━━━━━━━━━━━━━━━ 143s 13ms/step - accuracy: 

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_9             │ (None, 50)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_9 (Embedding)   │ (None, 50, 32)         │         64,000 │ input_layer_9[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_1 (Conv1D)         │ (None, 48, 128)        │         12,416 │ embedding_9[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_6           │ (None, 48, 128)        │         98,816 │ conv1d_1[0][0]         │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_3 (Attention)   │ (None, 48, 128)        │              0 │ bidirectional_6[0][0], │
│                           │                        │                │ bidirectional_6[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_9 (Dropout)       │ (None, 48, 128)        │              0 │ attention_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_13 (LSTM)            │ (None, 32)             │         20,608 │ dropout_9[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_10 (Dropout)      │ (None, 32)             │              0 │ lstm_13[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_12 (Dense)          │ (None, 16)             │            528 │ dropout_10[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_13 (Dense)          │ (None, 2)              │             34 │ dense_12[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 589,208 (2.25 MB)

 Trainable params: 196,402 (767.20 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 392,806 (1.50 MB)

# Hyperparameter Tuning


## Tuning 1

Batch size 16 -> 64

embedding dim 32 -> 64

max_len 50 -> 100


In [ ]:
#Since redefining these initial ones for tuning, have to re-tokenize as well
max_words = 2000
max_len = 100
embedding_dim = 64

#Tokenizing the text column
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data_processed['text'])
sequences = tokenizer.texts_to_sequences(data_processed['text'])

#Padding sequences to be same length
X = pad_sequences(sequences, maxlen=max_len)
y = np.array(data_processed['encoded_sentiment'])

#Input layer
input_layer = Input(shape=(max_len,))

#Embedding layer
embedding_layer = Embedding(max_words, embedding_dim, input_length=max_len)(input_layer)

#Convolutional Layer
conv_layer = Conv1D(filters=128, kernel_size=3, activation='relu')(embedding_layer)

#Bidirectional LSTM Layer
bidirectional_layer = Bidirectional(LSTM(64, return_sequences=True))(conv_layer)

#Attention Layer
attention_layer = Attention()([bidirectional_layer, bidirectional_layer])

#Dropout Layer
dropout_layer1 = Dropout(0.5)(attention_layer)

#LSTM Layer
lstm_layer2 = LSTM(32)(dropout_layer1)

#Dropout Layer
dropout_layer2 = Dropout(0.5)(lstm_layer2)

#Dense Layers
dense_layer = Dense(16, activation='relu')(dropout_layer2)
output_layer = Dense(2, activation='softmax')(dense_layer)

#Building the model
model = Model(inputs=input_layer, outputs=output_layer)

#Compiling the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Training the model
model.fit(X, y, epochs=10, batch_size=64, validation_split=0.2)

#Showing the Model summary
model.summary()

Epoch 1/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 40s 19ms/step - accuracy: 0.7007 - loss: 0.5557 - val_accuracy: 0.7738 - val_loss: 0.4713
Epoch 2/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 35s 18ms/step - accuracy: 0.7885 - loss: 0.4561 - val_accuracy: 0.7828 - val_loss: 0.4560
Epoch 3/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 42s 18ms/step - accuracy: 0.8000 - loss: 0.4317 - val_accuracy: 0.7884 - val_loss: 0.4517
Epoch 4/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 42s 18ms/step - accuracy: 0.8149 - loss: 0.4095 - val_accuracy: 0.7853 - val_loss: 0.4510
Epoch 5/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 41s 18ms/step - accuracy: 0.8248 - loss: 0.3902 - val_accuracy: 0.7827 - val_loss: 0.4560
Epoch 6/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 43s 19ms/step - accuracy: 0.8355 - loss: 0.3677 - val_accuracy: 0.7802 - val_loss: 0.4656
Epoch 7/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 37s 19ms/step - accuracy: 0.8472 - loss: 0.3456 - val_accuracy: 0.7783 - val_loss: 0.4746
Epoch 8/10
2000/2000 ━━━━━━━━━━━━━━━━━━━━ 40s 18ms/step - accuracy: 0.8579 -

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_11            │ (None, 100)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_11 (Embedding)  │ (None, 100, 64)        │        128,000 │ input_layer_11[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_3 (Conv1D)         │ (None, 98, 128)        │         24,704 │ embedding_11[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_8           │ (None, 98, 128)        │         98,816 │ conv1d_3[0][0]         │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_5 (Attention)   │ (None, 98, 128)        │              0 │ bidirectional_8[0][0], │
│                           │                        │                │ bidirectional_8[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_13 (Dropout)      │ (None, 98, 128)        │              0 │ attention_5[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_17 (LSTM)            │ (None, 32)             │         20,608 │ dropout_13[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_14 (Dropout)      │ (None, 32)             │              0 │ lstm_17[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_16 (Dense)          │ (None, 16)             │            528 │ dropout_14[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_17 (Dense)          │ (None, 2)              │             34 │ dense_16[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 818,072 (3.12 MB)

 Trainable params: 272,690 (1.04 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 545,382 (2.08 MB)

## Tuning 2

embedding dim 64 -> 128

conv filters 128 -> 256

LSTM1 64 -> 128

LSTM2 32 -> 64


In [ ]:
max_words = 2000
max_len = 100
embedding_dim = 128

#Tokenizing the text column
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data_processed['text'])
sequences = tokenizer.texts_to_sequences(data_processed['text'])

#Padding sequences to be same length
X = pad_sequences(sequences, maxlen=max_len)
y = np.array(data_processed['encoded_sentiment'])

#Input layer
input_layer = Input(shape=(max_len,))

#Embedding layer
embedding_layer = Embedding(max_words, embedding_dim, input_length=max_len)(input_layer)

#Convolutional Layer
conv_layer = Conv1D(filters=256, kernel_size=3, activation='relu')(embedding_layer)

#Bidirectinal LSTM Layer
bidirectional_layer = Bidirectional(LSTM(128, return_sequences=True))(conv_layer)

#Attention Layer
attention_layer = Attention()([bidirectional_layer, bidirectional_layer])

#Dropout Layer
dropout_layer1 = Dropout(0.5)(attention_layer)

#LSTM Layer
lstm_layer2 = LSTM(64)(dropout_layer1)

#Dropout Layer
dropout_layer2 = Dropout(0.5)(lstm_layer2)

#Dense Layers
dense_layer = Dense(16, activation='relu')(dropout_layer2)
output_layer = Dense(2, activation='softmax')(dense_layer)

#Building the model
model = Model(inputs=input_layer, outputs=output_layer)

#Compiling the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Training the model
model.fit(X, y, epochs=10, batch_size=128, validation_split=0.2)

#Showing the Model summary
model.summary()

Epoch 1/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 42s 38ms/step - accuracy: 0.6963 - loss: 0.5634 - val_accuracy: 0.7759 - val_loss: 0.4677
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 38s 35ms/step - accuracy: 0.7905 - loss: 0.4524 - val_accuracy: 0.7879 - val_loss: 0.4503
Epoch 3/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 44s 38ms/step - accuracy: 0.8045 - loss: 0.4250 - val_accuracy: 0.7880 - val_loss: 0.4469
Epoch 4/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 41s 38ms/step - accuracy: 0.8181 - loss: 0.3991 - val_accuracy: 0.7858 - val_loss: 0.4574
Epoch 5/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 38s 38ms/step - accuracy: 0.8271 - loss: 0.3779 - val_accuracy: 0.7835 - val_loss: 0.4604
Epoch 6/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 39s 36ms/step - accuracy: 0.8407 - loss: 0.3557 - val_accuracy: 0.7812 - val_loss: 0.4834
Epoch 7/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 41s 36ms/step - accuracy: 0.8561 - loss: 0.3249 - val_accuracy: 0.7735 - val_loss: 0.4940
Epoch 8/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 41s 36ms/step - accuracy: 0.8695 -

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_14            │ (None, 100)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_14 (Embedding)  │ (None, 100, 128)       │        256,000 │ input_layer_14[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_6 (Conv1D)         │ (None, 98, 256)        │         98,560 │ embedding_14[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_11          │ (None, 98, 256)        │        394,240 │ conv1d_6[0][0]         │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_8 (Attention)   │ (None, 98, 256)        │              0 │ bidirectional_11[0][0… │
│                           │                        │                │ bidirectional_11[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_19 (Dropout)      │ (None, 98, 256)        │              0 │ attention_8[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_23 (LSTM)            │ (None, 64)             │         82,176 │ dropout_19[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_20 (Dropout)      │ (None, 64)             │              0 │ lstm_23[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_22 (Dense)          │ (None, 16)             │          1,040 │ dropout_20[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_23 (Dense)          │ (None, 2)              │             34 │ dense_22[0][0]         │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 2,496,152 (9.52 MB)

 Trainable params: 832,050 (3.17 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,664,102 (6.35 MB)

## Tuning 3

embedding dim 128 -> 256

conv filters 256 -> 512

LSTM1 128 -> 256

LSTM2 64 -> 128

In [ ]:
max_words = 2000
max_len = 100
embedding_dim = 256

#Tokenizing the text column
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data_processed['text'])
sequences = tokenizer.texts_to_sequences(data_processed['text'])

#Padding sequences to be samelength
X = pad_sequences(sequences, maxlen=max_len)
y = np.array(data_processed['encoded_sentiment'])

#Input layer
input_layer = Input(shape=(max_len,))

#Embedding layer
embedding_layer = Embedding(max_words, embedding_dim, input_length=max_len)(input_layer)

#Convolutional Layer
conv_layer = Conv1D(filters=512, kernel_size=3, activation='relu')(embedding_layer)

#Bidirectional LSTM Layer
bidirectional_layer = Bidirectional(LSTM(256, return_sequences=True))(conv_layer)

#Attention Layer
attention_layer = Attention()([bidirectional_layer, bidirectional_layer])

#Dropout Layer
dropout_layer1 = Dropout(0.5)(attention_layer)

#LSTM LAyer
lstm_layer2 = LSTM(128)(dropout_layer1)

#Dropout Layer
dropout_layer2 = Dropout(0.5)(lstm_layer2)

#Dense Layers
dense_layer = Dense(16, activation='relu')(dropout_layer2)
output_layer = Dense(2, activation='softmax')(dense_layer)

#Building the model
model = Model(inputs=input_layer, outputs=output_layer)

#Compiling the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Training the model
model.fit(X, y, epochs=10, batch_size=128, validation_split=0.2)

#SHowing the Model summary
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 91s 83ms/step - accuracy: 0.6953 - loss: 0.5605 - val_accuracy: 0.7818 - val_loss: 0.4620
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 136s 82ms/step - accuracy: 0.7875 - loss: 0.4515 - val_accuracy: 0.7875 - val_loss: 0.4534
Epoch 3/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 82s 81ms/step - accuracy: 0.8038 - loss: 0.4246 - val_accuracy: 0.7845 - val_loss: 0.4520
Epoch 4/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 82s 81ms/step - accuracy: 0.8149 - loss: 0.4023 - val_accuracy: 0.7823 - val_loss: 0.4586
Epoch 5/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 86s 85ms/step - accuracy: 0.8291 - loss: 0.3790 - val_accuracy: 0.7841 - val_loss: 0.4616
Epoch 6/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 138s 82ms/step - accuracy: 0.8412 - loss: 0.3561 - val_accuracy: 0.7830 - val_loss: 0.4706
Epoch 7/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 86s 86ms/step - accuracy: 0.8552 - loss: 0.3298 - val_accuracy: 0.7806 - val_loss: 0.4909
Epoch 8/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 142s 86ms/step - accuracy: 0.871

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 100)            │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding (Embedding)     │ (None, 100, 256)       │        512,000 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d (Conv1D)           │ (None, 98, 512)        │        393,728 │ embedding[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional             │ (None, 98, 512)        │      1,574,912 │ conv1d[0][0]           │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention (Attention)     │ (None, 98, 512)        │              0 │ bidirectional[0][0],   │
│                           │                        │                │ bidirectional[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 98, 512)        │              0 │ attention[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ (None, 128)            │        328,192 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_1 (Dropout)       │ (None, 128)            │              0 │ lstm_1[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 16)             │          2,064 │ dropout_1[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 2)              │             34 │ dense[0][0]            │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 8,432,792 (32.17 MB)

 Trainable params: 2,810,930 (10.72 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 5,621,862 (21.45 MB)

## Tuning 4

Batch size 256 -> 512

embedding dim 256 -> 512

LSTM1 256 -> 512

LSTM2 128 -> 256

In [ ]:
max_words = 2000
max_len = 100
embedding_dim = 512

#Tokenizing the text column
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data_processed['text'])
sequences = tokenizer.texts_to_sequences(data_processed['text'])

#Padding sequences to be same length
X = pad_sequences(sequences, maxlen=max_len)
y = np.array(data_processed['encoded_sentiment'])

#Input layer
input_layer = Input(shape=(max_len,))

#Embedding layer
embedding_layer = Embedding(max_words, embedding_dim, input_length=max_len)(input_layer)

#Convolutional Layer
conv_layer = Conv1D(filters=512, kernel_size=3, activation='relu')(embedding_layer)

#Bidirectional LSTM Layer
bidirectional_layer = Bidirectional(LSTM(512, return_sequences=True))(conv_layer)

#Attention Layer
attention_layer = Attention()([bidirectional_layer, bidirectional_layer])

#Dropout LAyer
dropout_layer1 = Dropout(0.5)(attention_layer)

#LSTM Layer
lstm_layer2 = LSTM(256)(dropout_layer1)

#Dropout Layer
dropout_layer2 = Dropout(0.5)(lstm_layer2)

#Dense Layers
dense_layer = Dense(16, activation='relu')(dropout_layer2)
output_layer = Dense(2, activation='softmax')(dense_layer)

#Building the model
model = Model(inputs=input_layer, outputs=output_layer)

#Compiling the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Training the model
model.fit(X, y, epochs=10, batch_size=128, validation_split=0.2)

#Showing the Model summary
model.summary()

Epoch 1/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 183s 180ms/step - accuracy: 0.6979 - loss: 0.5609 - val_accuracy: 0.7803 - val_loss: 0.4626
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 209s 187ms/step - accuracy: 0.7878 - loss: 0.4497 - val_accuracy: 0.7867 - val_loss: 0.4528
Epoch 3/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 186s 186ms/step - accuracy: 0.8016 - loss: 0.4251 - val_accuracy: 0.7869 - val_loss: 0.4534
Epoch 4/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 196s 180ms/step - accuracy: 0.8185 - loss: 0.4006 - val_accuracy: 0.7879 - val_loss: 0.4573
Epoch 5/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 208s 186ms/step - accuracy: 0.8303 - loss: 0.3784 - val_accuracy: 0.7827 - val_loss: 0.4605
Epoch 6/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 195s 180ms/step - accuracy: 0.8479 - loss: 0.3444 - val_accuracy: 0.7804 - val_loss: 0.4855
Epoch 7/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 209s 186ms/step - accuracy: 0.8633 - loss: 0.3159 - val_accuracy: 0.7803 - val_loss: 0.5002
Epoch 8/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 186s 186ms/step - ac

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2             │ (None, 100)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_2 (Embedding)   │ (None, 100, 512)       │      1,024,000 │ input_layer_2[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_2 (Conv1D)         │ (None, 98, 512)        │        786,944 │ embedding_2[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_2           │ (None, 98, 1024)       │      4,198,400 │ conv1d_2[0][0]         │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_2 (Attention)   │ (None, 98, 1024)       │              0 │ bidirectional_2[0][0], │
│                           │                        │                │ bidirectional_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_4 (Dropout)       │ (None, 98, 1024)       │              0 │ attention_2[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_5 (LSTM)             │ (None, 256)            │      1,311,744 │ dropout_4[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_5 (Dropout)       │ (None, 256)            │              0 │ lstm_5[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 16)             │          4,112 │ dropout_5[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 2)              │             34 │ dense_4[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 21,975,704 (83.83 MB)

 Trainable params: 7,325,234 (27.94 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 14,650,470 (55.89 MB)

Out of these hyperparameter tunings, results are similar, but tuning 2 takes the least amount of computational power and still has some of the highest results, so settling on these parameters. One more thing to check is if max_len affects the accuracy.

## Tuning 5

max_len = 50



In [ ]:
max_words = 2000
max_len = 50
embedding_dim = 128

#Tokenizing the text column
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data_processed['text'])
sequences = tokenizer.texts_to_sequences(data_processed['text'])

#Padding sequences to be same length
X = pad_sequences(sequences, maxlen=max_len)
y = np.array(data_processed['encoded_sentiment'])

#Input layer
input_layer = Input(shape=(max_len,))

#Embedding layer
embedding_layer = Embedding(max_words, embedding_dim, input_length=max_len)(input_layer)

#Convolutional LAyer
conv_layer = Conv1D(filters=256, kernel_size=3, activation='relu')(embedding_layer)

#Bidirectional LSTM Layer
bidirectional_layer = Bidirectional(LSTM(128, return_sequences=True))(conv_layer)

#Attention Layer
attention_layer = Attention()([bidirectional_layer, bidirectional_layer])

#Dropout Layer
dropout_layer1 = Dropout(0.5)(attention_layer)

#LSTM Layer
lstm_layer2 = LSTM(64)(dropout_layer1)

#Dropout Layer
dropout_layer2 = Dropout(0.5)(lstm_layer2)

#Dense Layers
dense_layer = Dense(16, activation='relu')(dropout_layer2)
output_layer = Dense(2, activation='softmax')(dense_layer)

#Building the model
model = Model(inputs=input_layer, outputs=output_layer)

#Compiling the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Training the model
model.fit(X, y, epochs=10, batch_size=128, validation_split=0.2)

#Showing the Model summary
model.summary()

Epoch 1/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 24s 20ms/step - accuracy: 0.6993 - loss: 0.5531 - val_accuracy: 0.7766 - val_loss: 0.4635
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 40s 20ms/step - accuracy: 0.7882 - loss: 0.4517 - val_accuracy: 0.7878 - val_loss: 0.4520
Epoch 3/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 22s 21ms/step - accuracy: 0.8040 - loss: 0.4228 - val_accuracy: 0.7867 - val_loss: 0.4513
Epoch 4/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 41s 21ms/step - accuracy: 0.8172 - loss: 0.4025 - val_accuracy: 0.7852 - val_loss: 0.4557
Epoch 5/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 21s 21ms/step - accuracy: 0.8309 - loss: 0.3773 - val_accuracy: 0.7869 - val_loss: 0.4571
Epoch 6/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 21s 21ms/step - accuracy: 0.8431 - loss: 0.3549 - val_accuracy: 0.7821 - val_loss: 0.4723
Epoch 7/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 40s 20ms/step - accuracy: 0.8583 - loss: 0.3270 - val_accuracy: 0.7761 - val_loss: 0.4877
Epoch 8/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 21s 20ms/step - accuracy: 0.8717 -

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3             │ (None, 50)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_3 (Embedding)   │ (None, 50, 128)        │        256,000 │ input_layer_3[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_3 (Conv1D)         │ (None, 48, 256)        │         98,560 │ embedding_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_3           │ (None, 48, 256)        │        394,240 │ conv1d_3[0][0]         │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_3 (Attention)   │ (None, 48, 256)        │              0 │ bidirectional_3[0][0], │
│                           │                        │                │ bidirectional_3[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_6 (Dropout)       │ (None, 48, 256)        │              0 │ attention_3[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_7 (LSTM)             │ (None, 64)             │         82,176 │ dropout_6[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_7 (Dropout)       │ (None, 64)             │              0 │ lstm_7[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_6 (Dense)           │ (None, 16)             │          1,040 │ dropout_7[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_7 (Dense)           │ (None, 2)              │             34 │ dense_6[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 2,496,152 (9.52 MB)

 Trainable params: 832,050 (3.17 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,664,102 (6.35 MB)

## Tuning 6

max_len = 200

In [ ]:
max_words = 2000
max_len = 200
embedding_dim = 128

#Tokenizing the text column
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(data_processed['text'])
sequences = tokenizer.texts_to_sequences(data_processed['text'])

#Padding sequences to be same length
X = pad_sequences(sequences, maxlen=max_len)
y = np.array(data_processed['encoded_sentiment'])

#Input layer
input_layer = Input(shape=(max_len,))

#Embedding layer
embedding_layer = Embedding(max_words, embedding_dim, input_length=max_len)(input_layer)

#Convolutional Layer
conv_layer = Conv1D(filters=256, kernel_size=3, activation='relu')(embedding_layer)

#bidirectional LSTM Layer
bidirectional_layer = Bidirectional(LSTM(128, return_sequences=True))(conv_layer)

#Attentino Layer
attention_layer = Attention()([bidirectional_layer, bidirectional_layer])

#Dropout Layer
dropout_layer1 = Dropout(0.5)(attention_layer)

#LSTM Layer
lstm_layer2 = LSTM(64)(dropout_layer1)

#Dropout Layer
dropout_layer2 = Dropout(0.5)(lstm_layer2)

#Dense Layers
dense_layer = Dense(16, activation='relu')(dropout_layer2)
output_layer = Dense(2, activation='softmax')(dense_layer)

#Building the model
model = Model(inputs=input_layer, outputs=output_layer)

#Compiling the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#Training the model
model.fit(X, y, epochs=10, batch_size=128, validation_split=0.2)

#Showing the Model summary
model.summary()

Epoch 1/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 77s 73ms/step - accuracy: 0.6623 - loss: 0.5855 - val_accuracy: 0.7791 - val_loss: 0.4680
Epoch 2/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 73s 73ms/step - accuracy: 0.7892 - loss: 0.4533 - val_accuracy: 0.7853 - val_loss: 0.4536
Epoch 3/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 77s 68ms/step - accuracy: 0.8011 - loss: 0.4308 - val_accuracy: 0.7869 - val_loss: 0.4497
Epoch 4/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 82s 69ms/step - accuracy: 0.8171 - loss: 0.4029 - val_accuracy: 0.7883 - val_loss: 0.4509
Epoch 5/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 87s 74ms/step - accuracy: 0.8303 - loss: 0.3827 - val_accuracy: 0.7806 - val_loss: 0.4542
Epoch 6/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 77s 68ms/step - accuracy: 0.8429 - loss: 0.3571 - val_accuracy: 0.7861 - val_loss: 0.4798
Epoch 7/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 82s 69ms/step - accuracy: 0.8555 - loss: 0.3321 - val_accuracy: 0.7732 - val_loss: 0.4839
Epoch 8/10
1000/1000 ━━━━━━━━━━━━━━━━━━━━ 82s 69ms/step - accuracy: 0.8682 -

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_4             │ (None, 200)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_4 (Embedding)   │ (None, 200, 128)       │        256,000 │ input_layer_4[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_4 (Conv1D)         │ (None, 198, 256)       │         98,560 │ embedding_4[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_4           │ (None, 198, 256)       │        394,240 │ conv1d_4[0][0]         │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_4 (Attention)   │ (None, 198, 256)       │              0 │ bidirectional_4[0][0], │
│                           │                        │                │ bidirectional_4[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_8 (Dropout)       │ (None, 198, 256)       │              0 │ attention_4[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_9 (LSTM)             │ (None, 64)             │         82,176 │ dropout_8[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_9 (Dropout)       │ (None, 64)             │              0 │ lstm_9[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_8 (Dense)           │ (None, 16)             │          1,040 │ dropout_9[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_9 (Dense)           │ (None, 2)              │             34 │ dense_8[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 2,496,152 (9.52 MB)

 Trainable params: 832,050 (3.17 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,664,102 (6.35 MB)